# A tiny bot for NLE

In [1]:
import torch
import numpy

import matplotlib.pyplot as plt

import gym
import nle

<br>

## Understanding the actions

Collect the action info by using the `&` command.

In [2]:
with gym.make("NetHackChallenge-v0") as env:
    obs = env.reset()
    env.step(92)  # &
    env.step(19)  # \015 ENTER

    info = {}
    for j, a in enumerate(env.unwrapped._actions):
        obs, *_ = env.step(j)
        info[j] = int(a), a.__class__.__name__, a._name_, obs['message'].view('S256')[0].decode('utf8')
        while b'--More--' in bytes(obs['tty_chars']):
            env.step(36)  #\033 ESC 
        env.step(92)

Read the command help from the nle's docs.

In [3]:
cmdhelp = {}
with open('/Users/ivannazarov/Github/repos_with_rl/nle/dat/cmdhelp', 'tr') as fin:
    for line in fin:
        key, tab, val = line.strip().partition('\t')
        if not tab:
            continue
        
        cmdhelp[key] = val

The following actions will wither be emulated or totally unused by the agent.

In [4]:
unused_actions = {
# long actions skip too much game ticks!
      8,  # CompassDirectionLonger.N      75
      9,  # CompassDirectionLonger.E      76
     10,  # CompassDirectionLonger.S      74
     11,  # CompassDirectionLonger.W      72
     12,  # CompassDirectionLonger.NE     85
     13,  # CompassDirectionLonger.SE     78
     14,  # CompassDirectionLonger.SW     66
     15,  # CompassDirectionLonger.NW     89

# list of extendend commands
     21,  # Command.EXTLIST               191

# adjust inventory letters
     22,  # Command.ADJUST                225

# annotate a location on the map
     23,  # Command.ANNOTATE              193
     27,  # Command.CALL                  67

# inspect char's attributes
     25,  # Command.ATTRIBUTES            24
     76,  # Command.SEEALL                42
     59,  # Command.OVERVIEW              15

     26,  # Command.AUTOPICKUP            64
     31,  # Command.CONDUCT               195
     34,  # Command.DROPTYPE              68
     37,  # Command.ENGRAVE               69
     42,  # Command.GLANCE                59
     81,  # Command.TAKEOFFALL            65

# inventory is given to us
     44,  # Command.INVENTORY             105
     45,  # Command.INVENTTYPE            73

# we don't care about the logs
     49,  # Command.KNOWN                 92
     50,  # Command.KNOWNCLASS            96

# look us emulated through inventory probing
     51,  # Command.LOOK                  58

# some unclear navigation actions
     54,  # Command.MOVE                  109
     55,  # Command.MOVEFAR               77
     72,  # Command.RUSH                  103
     73,  # Command.RUSH2                 71
     85,  # Command.TRAVEL                95

# misc, developer
     89,  # Command.VERSION               246
     90,  # Command.VERSIONSHORT          118
     43,  # Command.HISTORY               86
     74,  # Command.SAVE                  83
     65,  # Command.QUIT                  241
     68,  # Command.REDRAW                18
     58,  # Command.OPTIONS               79

# command help
     92,  # Command.WHATDOES              38
     93,  # Command.WHATIS                47

# unknown
    100,  # TextCharacters.APOS           39
     99,  # TextCharacters.SPACE          20
     98,  # TextCharacters.MINUS          45
    102,  # TextCharacters.NUM_0          48
    103,  # TextCharacters.NUM_1          49
    104,  # TextCharacters.NUM_2          50
    105,  # TextCharacters.NUM_3          51
    106,  # TextCharacters.NUM_4          52
    107,  # TextCharacters.NUM_5          53
    108,  # TextCharacters.NUM_6          54
    109,  # TextCharacters.NUM_7          55
    110,  # TextCharacters.NUM_8          56
    111,  # TextCharacters.NUM_9          57

# can read the vloume of the purse from `blstats`.gold
    112,  # TextCharacters.DOLLAR         36

# these might possibly be useful
     47,  # Command.JUMP                  234
     70,  # Command.RIDE                  210  // mount or dismount a saddled steed.
     71,  # Command.RUB                   242  // Rub a lamp or a touchstone
     53,  # Command.MONSTER               237  // When polymorphed, use a monster's special ability
     60,  # Command.PAY                   112  // Pay your shopping bill
}

A neat table with oct, hex and descriptions

In [5]:
for j, (a, c, f, m) in info.items():
    if j in unused_actions:
        continue
    if m.startswith('No such command'):
        m = cmdhelp.get(chr(a))
    print(f"{j:>4d}  {c+'.'+f:28s}  {a:>03o}   {a:>02X}  {a if a > 32 else 46:>2c}  {m}")

   0  CompassDirection.N            153   6B   k  k       move north (screen up).
   1  CompassDirection.E            154   6C   l  l       move east (screen right).
   2  CompassDirection.S            152   6A   j  j       move south (screen down).
   3  CompassDirection.W            150   68   h  h       move west (screen left).
   4  CompassDirection.NE           165   75   u  u       move northeast (screen upper right).
   5  CompassDirection.SE           156   6E   n  n       move southeast (screen lower right).
   6  CompassDirection.SW           142   62   b  b       move southwest (screen lower left).
   7  CompassDirection.NW           171   79   y  y       move northwest (screen upper left).
  16  MiscDirection.UP              074   3C   <  <       go up a staircase.
  17  MiscDirection.DOWN            076   3E   >  >       go down a staircase.
  18  MiscDirection.WAIT            056   2E   .  .       rest one move while doing nothing.
  19  MiscAction.MORE               015 

<br>

Import level patching and options from `minihack`

In [6]:
import minihack
import minihack.envs.fightcorridor

from minihack.base import MiniHack, MH_NETHACKOPTIONS

from minihack.level_generator import LevelGenerator

lvl_gen = LevelGenerator(
    map="""
-----       ----------------------
|....#######.....................|
-----       ----------------------
""",
    lit=True,
)
lvl_gen.set_start_rect((1, 1), (3, 1))
# lvl_gen.add_monster(name="giant rat", place=(30, 1))
# lvl_gen.add_monster(name="giant rat", place=(29, 1))
lvl_gen.add_goal_pos((32, 1))



<br>

Some observation preprocessing

In [7]:
from nle_tools.utils.obs import BLStats, uint8_to_str

A renderer

In [8]:
from io import StringIO
from IPython.display import clear_output
from contextlib import redirect_stdout

def render(env, obs):
    blstats = BLStats(*obs['blstats'])

    with redirect_stdout(StringIO()) as f:
        env.render('human')

    screen = f.getvalue()
    screen += "\n" + (b''.join(obs['inv_letters'].view('c'))).decode()
    
    screen += f"\ntime: {blstats.time:04d}"
    screen += "\n" + str(obs['message'].view('S256')[0])

    screen += "\n" + str(obs['glyphs'][blstats.y-2:blstats.y+3, blstats.x-2:blstats.x+3,])

    clear_output(wait=False)
    print(screen)
    return True

Actors

In [9]:
class HumanActor:
    def override(self, obs):
        return False

    # human control
    def reset(self, obs):
        pass

    def step(self, obs):
        input = bytes(__builtins__.input('> '), 'utf8')
        yield from map(ord, input.decode('unicode-escape'))

Gather actions

In [10]:
import re

from collections import deque
from nle.nethack.actions import Command, MiscAction


class Skeleton:
    re_ynq = re.compile(br".*\?\s+\[[ynaq\d]{2,}\]", re.I)  # detect YN questions
    def __init__(self, brain, *, auto_more=True, auto_yesman=False):
        self.brain, self.queue = brain, deque()
        self.auto_more, self.auto_yesman = auto_more, auto_yesman

    def reset(self, obs):
        self.brain.reset(obs)
        self.queue.clear()

    def step(self, obs):
        # 1. automatic gui-related actions
        # skip partial info messages in the tty (`--More--`)
        # XXX `ESC` skips all messages, `ENTER` goes through them one-by-one
        if self.auto_more and b'--More--' in bytes(obs['tty_chars']):
            return Command.ESC  # MiscAction.MORE
        
        # 1.5 eternal internal `yes-man`: agree to every action we take
        if self.auto_yesman and self.re_ynq.match(bytes(obs['message'])):
            return ord('y')

        # 2. open/closed loop control. Prompt only if we are out of
        #  scheduled actions.
        if self.brain.override(obs):
            self.queue.clear()

        if not self.queue:
            # closed loop control: the brain tells us what to do next
            self.queue.extend(self.brain.step(obs))

        # open loop policy: execute pre-scheduled actions
        if self.queue:
            return self.queue.popleft()

        # just wait
        return ord('.')

Seed

In [11]:
from nle_tools.wrappers.replay import Replay

env = Replay(gym.make("NetHackChallenge-v0"))  #, options=MH_NETHACKOPTIONS))
# MiniHack._patch_nhdat(env.unwrapped, lvl_gen.get_des())
env.seed(123)

(12587170189557361100, 992822559630912802)

play
* `#pray\x0dy` -- pray to a diety and then die
* `#quit\015y` -- quit the game
* `DA\x0djlkh,,\x0d` -- drop all, walk a circle, pick up all

In [12]:
bot = Skeleton(brain=HumanActor())

obs, rew, fin, info = env.reset(), 0., False, None
bot.reset(obs)

# base.py#L382 maps gym action numbers to chars
ctoa = {a: j for j, a in enumerate(env.unwrapped._actions)}
while render(env, obs) and not fin:
    try:
        obs, rew, fin, info = env.step(ctoa[bot.step(obs)])
    except KeyError:
        break


Hello Agent, welcome to NetHack!  You are a lawful dwarven Cavewoman.           
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                          --------                              
                                          ....d@..                              
                                          |......|                              
                                          ...$...|                              
                                          |......-                              
                           

> ?


In [13]:
assert False

AssertionError: 

<br>

### A dangeorus freezing combination

```python
# well, this is awkward...
env.reset()
env.step(42)
env.step(51)
```

<br>

# Trunk